In [1]:
from signals import complex_signal, holding_signal
from datasets.kaggle import KaggleDataset
import pandas as pd
import numpy as np

np.set_printoptions(precision=4, suppress=True)


In [2]:
def get_price_date(
    start: pd.Timestamp, end: pd.Timestamp
) -> tuple[np.ndarray, np.ndarray]:  # -> price, date
    dataset = KaggleDataset("Daily")
    df = dataset.get_timerange(start, end)
    return df["close"].values, df["date"].values


price, date = get_price_date(pd.Timestamp("2018-1-1"), pd.Timestamp("2020-1-1"))

In [3]:
from bot import sim_bot

def eval_fn(params):
    hw1, hw2, hw3, hd1, hd2, hd3, hsf, lw1, lw2, lw3, ld1, ld2, ld3, lsf = params

    high_freq = complex_signal(price, hw1, hw2, hw3, int(hd1), int(hd2), int(hd3), hsf)
    low_freq = complex_signal(price, lw1, lw2, lw3, int(ld1), int(ld2), int(ld3), lsf)
    
    return sim_bot(holding_signal(high_freq, low_freq), price)

In [4]:
# Test eval # EMA is high freq, LMA is low freq

# W = 100
# eval_fn([0, 0, 1, 0, 0, W, 2/(W+1), 0, 1, 0, 0, W, 0, 0])
# [2.066281795501709, 1.7877318859100342, 1.834380865097046, 1.772376298904419, 1.6807661056518555, 1.9571168422698975, 1.6476001739501953, 2.068119764328003, 2.2466516494750977, 1.7766005992889404]
print(eval_fn(
    [
        0.034499,
        0.992755,
        0.431289,
        568.466155,
        638.848069,
        619.925735,
        0.33447,
        0.786569,
        0.411078,
        0.164846,
        581.086939,
        38.867306,
        361.510681,
        0.710678,
    ]
))

15268.523682239718


In [5]:
from algos.woa import WOA
from algos.pso import PSO

woa = PSO(
    eval_fn,
    [
        (0.01, 1),
        (0.01, 1),
        (0.01, 1),
        (2, len(price)),
        (2, len(price)),
        (2, len(price)),
        (0.01, 1),
        (0.01, 1),
        (0.01, 1),
        (0.01, 1),
        (2, len(price)),
        (2, len(price)),
        (2, len(price)),
        (0.01, 1),
    ],
    [3, 4, 5, 10, 11, 12],
)

woa.optimise()

print("best sol: ", woa.best_params)
print("best score: ", eval_fn(woa.best_params))

best sol:  [np.float64(0.5429975437411074), np.float64(0.40640417940103934), np.float64(0.5964841223855873), 687, 247, 708, 0.01, np.float64(0.5514779840533692), np.float64(0.7344327202483314), np.float64(0.18200371269065366), 34, 542, 437, np.float64(0.5709313178320661)]
best score:  10715.187549703345
